ALUNO: PEDRO NÍCOLLAS PEREIRA LEON LOPES

### 2. Classificação com K-Nearest Neighbors (KNN)

Construa um modelo KNN para prever a classe de um conjunto de dados (por exemplo, Wine).

Utilize validação cruzada para ajustar o número de vizinhos e avalie o desempenho com a métrica F1-Score.

In [1]:
'''
DATASET UTILIZADO -> Wine Spain (data/wines/SPA.csv)

Disponível em: https://www.kaggle.com/datasets/fedesoriano/spanish-wine-quality-dataset/
'''

'\nDATASET UTILIZADO -> Wine Spain (data/wines/SPA.csv)\n\nDisponível em: https://www.kaggle.com/datasets/fedesoriano/spanish-wine-quality-dataset/\n'

In [18]:
# Conjunto de Importações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [3]:
dados = pd.read_csv("data/wines_SPA.csv")            # lendo o .csv

In [4]:
dados.head()            # vendo o cabeçalho do .csv

,winery,wine,year,rating,num_reviews,country,region,price,type,body,acidity
0,Teso La Monja,Tinto,2013,4.9,58,Espana,Toro,995.00,Toro Red,5.0,3.0
1,Artadi,Vina El Pison,2018,4.9,31,Espana,Vino de Espana,313.50,Tempranillo,4.0,2.0
2,Vega Sicilia,Unico,2009,4.8,1793,Espana,Ribera del Duero,324.95,Ribera Del Duero Red,5.0,3.0
3,Vega Sicilia,Unico,1999,4.8,1705,Espana,Ribera del Duero,692.96,Ribera Del Duero Red,5.0,3.0
4,Vega Sicilia,Unico,1996,4.8,1309,Espana,Ribera del Duero,778.06,Ribera Del Duero Red,5.0,3.0


In [5]:
dados.isna().sum()

winery            0
wine              0
year              2
rating            0
num_reviews       0
country           0
region            0
price             0
type            545
body           1169
acidity        1169
dtype: int64

In [6]:
dados = dados.dropna()          # drop nos NaN's
# dados.isna().sum()

In [7]:
dados['type'].unique()              # valores únicos para "tipo" de vinhos

array(['Toro Red', 'Tempranillo', 'Ribera Del Duero Red', 'Pedro Ximenez',
       'Red', 'Sherry', 'Priorat Red', 'Rioja Red', 'Rioja White',
       'Grenache', 'Cava', 'Verdejo', 'Syrah', 'Monastrell', 'Mencia',
       'Sparkling', 'Montsant Red', 'Albarino', 'Chardonnay',
       'Cabernet Sauvignon', 'Sauvignon Blanc'], dtype=object)

In [8]:
# Convertendo a variável categórica winery e region para numérica
from sklearn.preprocessing import LabelEncoder, label_binarize          # Importação do sklearn para transformação da variável (encoding)

encoder = LabelEncoder()           # chamando método LabelEncoder()

dados['winery'] = encoder.fit_transform(dados['winery'])            # tranformação de STR para NUM
dados['region'] = encoder.fit_transform(dados['region'])            # tranformação de STR para NUM
dados['wine'] = encoder.fit_transform(dados['wine'])                # tranformação de STR para NUM
# dados['country'] = encoder.fit_transform(dados['country'])          # transformação de STR para NUM (Country não está sendo dropado porque não consigo) -> dropada na hora de usar
dados['year'] = encoder.fit_transform(dados['year'])                # tranformação de STR para NUM

dados.head()                                                        # cabeçalho do .csv (Winery e region agora foram normalizado, com números escolhidos pelo algoritmo)

,winery,wine,year,rating,num_reviews,country,region,price,type,body,acidity
0,371,667,61,4.9,58,0,61,995.00,Toro Red,5.0,3.0
1,30,714,66,4.9,31,0,64,313.50,Tempranillo,4.0,2.0
2,395,683,57,4.8,1793,0,49,324.95,Ribera Del Duero Red,5.0,3.0
3,395,683,47,4.8,1705,0,49,692.96,Ribera Del Duero Red,5.0,3.0
4,395,683,44,4.8,1309,0,49,778.06,Ribera Del Duero Red,5.0,3.0


NORMALIZAÇÃO DOS DADOS

In [9]:
from sklearn.preprocessing import MinMaxScaler          # importação do sklearn para usar MinMax

In [10]:
normalizador = MinMaxScaler(feature_range=(0,1))            # função MinMax para criação de um normalizador

In [11]:
colunas_para_normalizar = ['rating', 'num_reviews', 'price', 'body', 'acidity',]

# Normalização dos valores para entre -1 e 1
for col in colunas_para_normalizar:
    dados[col] = normalizador.fit_transform(dados[col].values.reshape(-1, 1))

dados.head()

,winery,wine,year,rating,num_reviews,country,region,price,type,body,acidity
0,371,667,61,1.000000,0.001012,0,61,0.317913,Toro Red,1.000000,1.0
1,30,714,66,1.000000,0.000184,0,64,0.099069,Tempranillo,0.666667,0.5
2,395,683,57,0.857143,0.054235,0,49,0.102746,Ribera Del Duero Red,1.000000,1.0
3,395,683,47,0.857143,0.051535,0,49,0.220922,Ribera Del Duero Red,1.000000,1.0
4,395,683,44,0.857143,0.039388,0,49,0.248249,Ribera Del Duero Red,1.000000,1.0


Separando em amostra de treino e teste

In [12]:
from sklearn.model_selection import train_test_split            # importação do "train_test_split" do sklearn para realizar o treinamento

In [14]:
# Atribuindo valores para X e Y
X = dados.drop(['country', 'type'], axis=1).values  # ('Purchased' é a coluna alvo)
Y = dados['type'].values  # Variável alvo

In [20]:
X_treino,X_teste,Y_treino,Y_teste=train_test_split(X,Y,test_size=0.30,random_state=0)           # separação de 30% para teste e 70% para treino

Aplicando modelo KNN (K-Nearest Neighbors)

In [21]:
from sklearn.neighbors import KNeighborsClassifier

In [22]:
modelo_KNN = KNeighborsClassifier()

In [23]:
modelo_KNN.fit(X_treino, Y_treino)
knn_pred = modelo_KNN.predict(X_teste)
knn_acc = accuracy_score(Y_teste, knn_pred)
print("The training accuracy for KNN is:", round(modelo_KNN.score(X_treino, Y_treino)*100, 2), "%")
print("The testing accuracy for KNN is:", round(knn_acc * 100,2), "%")

The training accuracy for KNN is: 93.43 %
The testing accuracy for KNN is: 89.1 %
